# PUBG Prediction
2019.01.04 <br>
Damien Park


## 1. 라이브러리 및 데이터 불러오기(Importing library and Loading Data)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import keras
from keras.layers.core import Dense
from keras.layers.normalization import BatchNormalization

In [ ]:
train = pd.read_csv("../input/train_V2.csv")

---

## 2. 데이터 미리보기(Exploratory Data Analysis)

In [ ]:
train.head()

### 2-1. 데이터 요약(Describe)

In [ ]:
train.describe().T

### 2-2. 칼럼의 이름 및 타입(Column's name and Type)

In [ ]:
pd.DataFrame(train.dtypes, columns=["Type"])

---

## 3. 데이터 전처리(Data Preprocessing)

### 3-1. 이산형 데이터(Discrete variable) 

In [ ]:
print(list(train.columns[train.dtypes == "O"]))

Id <br>
게임플레이어에 할당되는 고유 값<br>
Unique value for game player

In [ ]:
print("Number of record:", len(train), "\nNumber of Unique Id:", len(pd.unique(train.Id)))

groupId <br>
duo, squad의 경우 할당되는 그룹 고유 값, solo의 경우 groupId가 각자 할당된다. <br>
같은 groupId에서는 같은 winPlacePerc가 할당된다.

matchId <br>
각 게임에 할당되는 고유 값

In [ ]:
print("Number of match: ", len(pd.unique(train.matchId)), "\nNumber of match(<9): ", sum(train.groupby("matchId").size() < 9))

보통 한 게임당 100명에서 80명 정도 참가하지만 몇몇 게임에서는 그렇지 않음

In [ ]:
temp = train.loc[train.matchId.isin(train.groupby("matchId").size()[train.groupby("matchId").size() < 9].index), :]
temp.loc[temp.matchId == "e263f4a227313a"]

게임참가인원 정보를 새로 생성한다. 

In [ ]:
temp = pd.DataFrame(train.groupby("matchId").size(), columns=["player"])
temp.reset_index(level=0, inplace=True)

In [ ]:
train = train.merge(temp, left_on="matchId", right_on="matchId")

matchType<br>
matchType에는 총 16가지의 타입이 있다. 크게 solo, duo, squad와 같은 그룹인원에 따른 구분과 fpp, tpp와 같은 플레이 시점에 따른 구분이 있다. <br>
그룹인원에 따라서 matchType_1이라는 새로운 칼럼을 생성하고  matchType_2에는 플레이 시점에 따라서 나눈정보를 새로 생성한다. 

In [ ]:
print("Type: ", pd.unique(train.matchType), "\nCount: ", len(pd.unique(train.matchType)))

In [ ]:
#게임인원별 분류(Division by number of player in group)
train["matchType_1"] = "-"
train.loc[(train.matchType == "solo-fpp") | 
          (train.matchType == "solo") | 
          (train.matchType == "normal-solo-fpp") | 
          (train.matchType == "normal-solo"), "matchType_1"] = "solo"

train.loc[(train.matchType == "duo-fpp") | 
          (train.matchType == "duo") | 
          (train.matchType == "normal-duo-fpp") | 
          (train.matchType == "normal-duo"), "matchType_1"] = "duo"

train.loc[(train.matchType == "squad-fpp") | 
          (train.matchType == "squad") | 
          (train.matchType == "normal-squad-fpp") | 
          (train.matchType == "normal-squad"), "matchType_1"] = "squad"

train.loc[(train.matchType == "flarefpp") | 
          (train.matchType == "flaretpp") | 
          (train.matchType == "crashfpp") | 
          (train.matchType == "crashtpp"), "matchType_1"] = "etc"

In [ ]:
# 게임시점별 분류(Division by viewpoint)
train["matchType_2"] = "-"
train.loc[(train.matchType == "solo-fpp") | 
          (train.matchType == "duo-fpp") | 
          (train.matchType == "squad-fpp") | 
          (train.matchType == "normal-solo-fpp") | 
          (train.matchType == "normal-duo-fpp") | 
          (train.matchType == "normal-squad-fpp") | 
          (train.matchType == "crashfpp") | 
          (train.matchType == "flarefpp"), "matchType_2"] = "fpp"

train.loc[(train.matchType == "solo") | 
          (train.matchType == "duo") | 
          (train.matchType == "squad") | 
          (train.matchType == "normal-solo") | 
          (train.matchType == "normal-duo") | 
          (train.matchType == "normal-squad") | 
          (train.matchType == "crashtpp") | 
          (train.matchType == "flaretpp"), "matchType_2"] = "tpp"

one-hot-encoding

In [ ]:
train["solo"] = 0
train["duo"] = 0
train["squad"] = 0
train["etc"] = 0

train.loc[train.matchType_1 == "solo", "solo"] = 1
train.loc[train.matchType_1 == "duo", "duo"] = 1
train.loc[train.matchType_1 == "squad", "squad"] = 1
train.loc[train.matchType_1 == "etc", "etc"] = 1

In [ ]:
train["fpp"] = 0
train["tpp"] = 0

train.loc[train.matchType_2 == "fpp", "fpp"] = 1
train.loc[train.matchType_2 == "tpp", "tpp"] = 1

### 3-2. 연속형 데이터(Continuous variable)

In [ ]:
print(list(train.columns[train.dtypes != "O"]))

In [ ]:
feature = ["assists", "boosts", "damageDealt", "DBNOs", "headshotKills", "heals", 
           "killPlace", "killPoints", "kills", "killStreaks", "longestKill", 
           "matchDuration", "maxPlace", "rankPoints", "revives", "rideDistance", 
           "roadKills", "swimDistance", "teamKills", "vehicleDestroys", "walkDistance", "weaponsAcquired", "winPoints", "player"]

In [ ]:
feature_1 = ["matchId", "assists", "boosts", "damageDealt", "DBNOs", "headshotKills", "heals", 
             "killPlace", "killPoints", "kills", "killStreaks", "longestKill", 
             "revives", "rideDistance", "roadKills", "swimDistance", "teamKills", 
             "vehicleDestroys", "walkDistance", "weaponsAcquired", "winPoints"]

In [ ]:
feature_2 = ["matchDuration", "maxPlace", "rankPoints", "player", "fpp", "tpp", "matchType_1"]

Check The NA value

In [ ]:
for i in list(train.columns[train.dtypes != "O"]):
    print(i, ":", sum(train[i].isna()))

#### winPlacePerc

In [ ]:
np.sum(train.winPlacePerc.isna())

There is only one missing value in winPlacePerc <br>
So, we have to remove it

In [ ]:
train = train.loc[train.winPlacePerc.notna(), :]

---

### 3-2-2. 정규화(Normalization)

#### 게임 별로 정규화
Normalizaton by matchId

In [ ]:
train.set_index("Id", inplace=True)
train.index.name = "Id"

In [ ]:
temp_1 = train.loc[:, feature_1]
temp_2 = train.loc[:, feature_2]

In [ ]:
def minmax(attr):
    return (attr - min(attr)) / (max(attr) - min(attr))

In [ ]:
temp_1 = temp_1.groupby("matchId").transform(minmax)
temp_2 = temp_2.groupby("matchType_1").transform(minmax)

# for i in temp_2.columns[:4]:
#     temp_2[i] = (temp_2[i] - min(temp_2[i])) / (max(temp_2[i]) - min(temp_2[i]))

In [ ]:
temp_1.fillna(value=0, inplace=True)
temp_2.fillna(value=0, inplace=True)

In [ ]:
X = pd.merge(temp_1, temp_2, on="Id")
X = pd.merge(X, train.loc[:, ["matchType_1", "winPlacePerc"]], on="Id")

In [ ]:
del temp_1, temp_2
X = X.reset_index()

---

## 6. 예측(Prediction)

### 6-1. 모델생성(Model Generation)

게임 인원별로 다른 모델을 만들어보자 <br>
Make model by matchtype

In [ ]:
print("Name: ", feature, "\nCount: ", len(feature))

In [ ]:
list_feat = ["assists", "boosts", "damageDealt", "DBNOs", "headshotKills", "heals", 
             "killPlace", "killPoints", "kills", "killStreaks", "longestKill", 
             "matchDuration", "maxPlace", "rankPoints", "revives", "rideDistance", 
             "roadKills", "swimDistance", "teamKills", "vehicleDestroys", "walkDistance", 
             "weaponsAcquired", "winPoints", "player", "fpp", "tpp"]

In [ ]:
list_feat_1 = ["assists", "boosts", "damageDealt", "DBNOs", "headshotKills", "heals", 
               "killPlace", "killPoints", "kills", "killStreaks", "longestKill", 
               "matchDuration", "maxPlace", "rankPoints", "revives", "rideDistance", 
               "roadKills", "swimDistance", "teamKills", "vehicleDestroys", "walkDistance", 
               "weaponsAcquired", "winPoints", "player", "fpp", "tpp", "matchId"]

In [ ]:
# 모델 1(solo)
model_1 = keras.models.Sequential()

model_1.add(Dense(32, input_dim=len(list_feat), activation="elu", kernel_initializer="he_normal"))
model_1.add(Dense(64, activation="elu", kernel_initializer="he_normal"))
model_1.add(Dense(128, activation="elu", kernel_initializer="he_normal"))
model_1.add(keras.layers.Dropout(0.25))

model_1.add(Dense(256, activation="elu", kernel_initializer="he_normal"))
model_1.add(Dense(256, activation="elu", kernel_initializer="he_normal"))
model_1.add(keras.layers.Dropout(0.25))

model_1.add(Dense(128, activation="elu", kernel_initializer="he_normal"))
model_1.add(Dense(64, activation="elu", kernel_initializer="he_normal"))
model_1.add(Dense(32, activation="elu", kernel_initializer="he_normal"))
model_1.add(keras.layers.Dropout(0.25))

model_1.add(Dense(1, activation="sigmoid"))

model_1.compile(optimizer="RMSprop", loss='MAE', metrics=["MAE"])

In [ ]:
x_train = X.loc[X.matchType_1 == "solo", list_feat]
y_train = X.loc[X.matchType_1 == "solo", ["winPlacePerc"]]

In [ ]:
model_1.fit(x=x_train, y=y_train, epochs=50, batch_size=2000, validation_split=0.2, shuffle=True)

In [ ]:
model_1.save("model_1_solo.h5")

In [ ]:
# 모델 2(duo)
model_2 = keras.models.Sequential()

model_2.add(Dense(32, input_dim=len(list_feat), activation="elu", kernel_initializer="he_normal"))
model_2.add(Dense(64, activation="elu", kernel_initializer="he_normal"))
model_2.add(Dense(128, activation="elu", kernel_initializer="he_normal"))
model_2.add(keras.layers.Dropout(0.25))

model_2.add(Dense(256, activation="elu", kernel_initializer="he_normal"))
model_2.add(Dense(256, activation="elu", kernel_initializer="he_normal"))
model_2.add(keras.layers.Dropout(0.25))

model_2.add(Dense(128, activation="elu", kernel_initializer="he_normal"))
model_2.add(Dense(64, activation="elu", kernel_initializer="he_normal"))
model_2.add(Dense(32, activation="elu", kernel_initializer="he_normal"))
model_2.add(keras.layers.Dropout(0.25))

model_2.add(Dense(1, activation="sigmoid"))

model_2.compile(optimizer="RMSprop", loss='MAE', metrics=["MAE"])

In [ ]:
x_train = X.loc[X.matchType_1 == "duo", list_feat]
y_train = X.loc[X.matchType_1 == "duo", ["winPlacePerc"]]

In [ ]:
model_2.fit(x=x_train, y=y_train, epochs=75, batch_size=2000, validation_split=0.2, shuffle=True)

In [ ]:
model_2.save("model_2_duo.h5")

In [ ]:
# 모델 3(squad)
model_3 = keras.models.Sequential()

model_3.add(Dense(32, input_dim=len(list_feat), activation="elu", kernel_initializer="he_normal"))
model_3.add(Dense(64, activation="elu", kernel_initializer="he_normal"))
model_3.add(Dense(128, activation="elu", kernel_initializer="he_normal"))
model_3.add(keras.layers.Dropout(0.25))

model_3.add(Dense(256, activation="elu", kernel_initializer="he_normal"))
model_3.add(Dense(256, activation="elu", kernel_initializer="he_normal"))
model_3.add(keras.layers.Dropout(0.35))

model_3.add(Dense(128, activation="elu", kernel_initializer="he_normal"))
model_3.add(Dense(64, activation="elu", kernel_initializer="he_normal"))
model_3.add(Dense(32, activation="elu", kernel_initializer="he_normal"))
model_3.add(keras.layers.Dropout(0.25))

model_3.add(Dense(1, activation="sigmoid"))

model_3.compile(optimizer="RMSprop", loss='MAE', metrics=["MAE"])

In [ ]:
x_train = X.loc[X.matchType_1 == "squad", list_feat]
y_train = X.loc[X.matchType_1 == "squad", ["winPlacePerc"]]

In [ ]:
model_3.fit(x=x_train, y=y_train, epochs=25, batch_size=3000, validation_split=0.2, shuffle=True)

In [ ]:
model_3.save("model_3_squad.h5")

In [ ]:
# 모델 4(etc)
model_4 = keras.models.Sequential()

model_4.add(Dense(32, input_dim=len(list_feat), activation="elu", kernel_initializer="he_normal"))
model_4.add(Dense(64, activation="elu", kernel_initializer="he_normal"))
model_4.add(Dense(128, activation="elu", kernel_initializer="he_normal"))
model_4.add(keras.layers.Dropout(0.25))

model_4.add(Dense(128, activation="elu", kernel_initializer="he_normal"))
model_4.add(Dense(64, activation="elu", kernel_initializer="he_normal"))
model_4.add(Dense(32, activation="elu", kernel_initializer="he_normal"))
model_4.add(keras.layers.Dropout(0.25))

model_4.add(Dense(1, activation="sigmoid"))

model_4.compile(optimizer="RMSprop", loss='MAE', metrics=["MAE"])

In [ ]:
x_train = X.loc[X.matchType_1 == "etc", list_feat]
y_train = X.loc[X.matchType_1 == "etc", ["winPlacePerc"]]

In [ ]:
model_4.fit(x=x_train, y=y_train, epochs=100, validation_split=0.2, shuffle=True)

In [ ]:
model_4.save("model_4_etc.h5")
# model_1 = keras.models.load_model("../input/model-pubg/model_1_fpp.h5")

In [ ]:
del(train, x_train, y_train, X)

### 6-2. 생성모델 평가

In [ ]:
plt.figure(figsize=(30, 15))
plt.suptitle("model History", fontsize = 20)

plt.subplot(1, 2, 1)
plt.title("model_1")
plt.plot(model_1.history.history["mean_absolute_error"], label="training")
plt.plot(model_1.history.history["val_mean_absolute_error"], label="validation")
plt.axhline(0.1, c="red", linestyle="--")
plt.axhline(0.05, c="yellow", linestyle="--")
plt.axhline(0.025, c="green", linestyle="--")
plt.xticks(model_1.history.epoch)
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.legend()

plt.subplot(1, 2, 2)
plt.title("model_2")
plt.plot(model_2.history.history["mean_absolute_error"], label="training")
plt.plot(model_2.history.history["val_mean_absolute_error"], label="validation")
plt.axhline(0.1, c="red", linestyle="--")
plt.axhline(0.05, c="yellow", linestyle="--")
plt.axhline(0.025, c="green", linestyle="--")
plt.xticks(model_2.history.epoch)
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(30, 15))
plt.suptitle("model History", fontsize = 20)

plt.subplot(1, 2, 1)
plt.title("model_3")
plt.plot(model_3.history.history["mean_absolute_error"], label="training")
plt.plot(model_3.history.history["val_mean_absolute_error"], label="validation")
plt.axhline(0.1, c="red", linestyle="--")
plt.axhline(0.05, c="yellow", linestyle="--")
plt.axhline(0.025, c="green", linestyle="--")
plt.xticks(model_3.history.epoch)
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.legend()

plt.subplot(1, 2, 2)
plt.title("model_4")
plt.plot(model_4.history.history["mean_absolute_error"], label="training")
plt.plot(model_4.history.history["val_mean_absolute_error"], label="validation")
plt.axhline(0.1, c="red", linestyle="--")
plt.axhline(0.05, c="yellow", linestyle="--")
plt.axhline(0.025, c="green", linestyle="--")
plt.xticks(model_4.history.epoch)
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.legend()

plt.show()

### 6-3. 예측 데이터 준비

In [ ]:
test = pd.read_csv("../input/test_V2.csv")

In [ ]:
print("Check The NA value in test data")
for i in list(test.columns[test.dtypes != "O"]):
    print(i, ":", sum(test[i].isna()))

Game Player

In [ ]:
len(pd.unique(test.matchId)), sum(test.groupby("matchId").size() < 9)

In [ ]:
temp = pd.DataFrame(test.groupby("matchId").size(), columns=["player"])
temp.reset_index(level=0, inplace=True)
test = test.merge(temp, left_on="matchId", right_on="matchId")

Division matchType

In [ ]:
test["matchType_1"] = "-"
test.loc[(test.matchType == "solo-fpp") | 
         (test.matchType == "solo") | 
         (test.matchType == "normal-solo-fpp") | 
         (test.matchType == "normal-solo"), "matchType_1"] = "solo"

test.loc[(test.matchType == "duo-fpp") | 
         (test.matchType == "duo") | 
         (test.matchType == "normal-duo-fpp") | 
         (test.matchType == "normal-duo"), "matchType_1"] = "duo"

test.loc[(test.matchType == "squad-fpp") | 
         (test.matchType == "squad") | 
         (test.matchType == "normal-squad-fpp") | 
         (test.matchType == "normal-squad"), "matchType_1"] = "squad"

test.loc[(test.matchType == "flarefpp") | 
         (test.matchType == "flaretpp") | 
         (test.matchType == "crashfpp") | 
         (test.matchType == "crashtpp"), "matchType_1"] = "etc"

In [ ]:
test["matchType_2"] = "-"
test.loc[(test.matchType == "solo-fpp") | 
         (test.matchType == "duo-fpp") | 
         (test.matchType == "squad-fpp") | 
         (test.matchType == "normal-solo-fpp") | 
         (test.matchType == "normal-duo-fpp") | 
         (test.matchType == "normal-squad-fpp") | 
         (test.matchType == "crashfpp") | 
         (test.matchType == "flarefpp"), "matchType_2"] = "fpp"

test.loc[(test.matchType == "solo") | 
         (test.matchType == "duo") | 
         (test.matchType == "squad") | 
         (test.matchType == "normal-solo") | 
         (test.matchType == "normal-duo") | 
         (test.matchType == "normal-squad") | 
         (test.matchType == "crashtpp") | 
         (test.matchType == "flaretpp"), "matchType_2"] = "tpp"

one-hot-encoding

In [ ]:
test["solo"] = 0
test["duo"] = 0
test["squad"] = 0
test["etc"] = 0

test.loc[test.matchType_1 == "solo", "solo"] = 1
test.loc[test.matchType_1 == "duo", "duo"] = 1
test.loc[test.matchType_1 == "squad", "squad"] = 1
test.loc[test.matchType_1 == "etc", "etc"] = 1

In [ ]:
test["fpp"] = 0
test["tpp"] = 0

test.loc[test.matchType_2 == "fpp", "fpp"] = 1
test.loc[test.matchType_2 == "tpp", "tpp"] = 1

정규화(Normalization)

In [ ]:
test.set_index("Id", inplace=True)
test.index.name = "Id"

In [ ]:
temp_1 = test.loc[:, feature_1]
temp_2 = test.loc[:, feature_2]

In [ ]:
temp_1 = temp_1.groupby("matchId").transform(minmax)
temp_2 = temp_2.groupby("matchType_1").transform(minmax)

# for i in temp_2.columns[:4]:
#     temp_2[i] = (temp_2[i] - min(temp_2[i])) / (max(temp_2[i]) - min(temp_2[i]))

In [ ]:
temp_1.fillna(value=0, inplace=True)
temp_2.fillna(value=0, inplace=True)

In [ ]:
X = pd.merge(temp_1, temp_2, on="Id")
X = pd.merge(X, test.loc[:, ["matchType_1", "winPlacePerc"]], on="Id")

In [ ]:
del temp_1, temp_2
X.reset_index()

### 6-4. 예측실행

In [ ]:
test = X
test = test.reset_index()

In [ ]:
solo = test[test.matchType_1 == "solo"].index
duo = test[test.matchType_1 == "duo"].index
squad = test[test.matchType_1 == "squad"].index
etc = test[test.matchType_1 == "etc"].index

solo = solo.append([duo, squad, etc])

temp = pd.DataFrame(index=solo)
temp.index.name = "Id"

In [ ]:
result_1 = model_1.predict(test.loc[test.matchType_1 == "solo", list_feat])
result_2 = model_2.predict(test.loc[test.matchType_1 == "duo", list_feat])
result_3 = model_3.predict(test.loc[test.matchType_1 == "squad", list_feat])
result_4 = model_4.predict(test.loc[test.matchType_1 == "etc", list_feat])

In [ ]:
temp = pd.DataFrame(test.loc[test.matchType_1 == "solo", "Id"]).append(pd.DataFrame(test.loc[test.matchType_1 == "duo", "Id"])).append(pd.DataFrame(test.loc[test.matchType_1 == "squad", "Id"])).append(pd.DataFrame(test.loc[test.matchType_1 == "etc", "Id"]))
_ = pd.DataFrame(result_1, columns = ["winPlacePerc"]).append(pd.DataFrame(result_2, columns = ["winPlacePerc"])).append(pd.DataFrame(result_3, columns = ["winPlacePerc"])).append(pd.DataFrame(result_4, columns = ["winPlacePerc"]))

In [ ]:
result = pd.concat([temp.reset_index(drop=True), _.reset_index(drop=True)], axis=1)

In [ ]:
print(np.sum(result.winPlacePerc.isna()), np.sum(result.winPlacePerc < 0), np.sum(result.winPlacePerc > 1))

In [ ]:
result.loc[result.winPlacePerc.isna(), "winPlacePerc"] = 0
result.loc[result.winPlacePerc < 0, "winPlacePerc"] = 0
result.loc[result.winPlacePerc > 1, "winPlacePerc"] = 1
result.to_csv('submission.csv', index=False)

---

The end of notebook

winner winner chicken dinner